In [ ]:
!pip install wikipedia pymorphy2 textdistance -q

# Семинар 1. Базовый пайплайн работы с текстом. Создания простого спеллчекера с помощью NLTK, векторов и рассстояния Левенштейна.

![picture](https://drive.google.com/uc?export=view&id=1Dh0iSF2AM-ApFQoMLigmDDBaNO978EAi)


## 1. Сбор данных

Под основные задачи было придумано огромное количество датасетов: [тональность](https://www.kaggle.com/datasets/abhi8923shriv/sentiment-analysis-dataset), [NER](https://github.com/juand-r/entity-recognition-datasets), [QA](https://rajpurkar.github.io/SQuAD-explorer/). Но... большинство данных на английском:)


### 1.1 Готовые корпуса

In [ ]:
import nltk
nltk.download('book')
nltk.download('all-corpora')

In [ ]:
from nltk.book import text1

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [ ]:
len(text1)

260819

In [ ]:
# text1[:100]

In [ ]:
text1.concordance('red')

Displaying 25 of 40 matches:
t did those aboriginal whalemen , the Red - Men , first sally out in canoes to 
 there . Further on , from the bright red windows of the " Sword - Fish Inn ," 
 Euroclydon ! says old Dives , in his red silken wrapper --( he had a redder on
w Bedford , they bloom like their own red roses . But roses only bloom in summe
ed the chapel with a handsome pair of red worsted man - ropes for this ladder ,
of how this island was settled by the red - men . Thus goes the legend . In old
n the porch of the inn , under a dull red lamp swinging there , that looked muc
 Quohog ! spring , thou chap with the red whiskers ; spring there , Scotch - ca
ists the last remnant of a village of red men , which has long supplied the nei
him from his mood . For , as when the red - cheeked , dancing girls , April and
e benignity of age ; though among the Red Men of America the giving of the whit
hen , with tornado brow , and eyes of red murder , and foam - glued lips , Ahab
ir beaches 

In [ ]:
from nltk.corpus import *

In [ ]:
?nltk.corpus

In [ ]:
# nltk.corpus.names.words('female.txt')

Ну и конечно, наше любимое, это стоп-слова.

In [ ]:
nltk.download("stopwords")
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")
russian_stopwords[:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

Существует и множество других встроенных библиотек с текстами (к примеру sklearn). Корпуса есть и на русском. Однако не редко встроенных текстов не достаточно и приходится пользоваться парсингом.

Давайте подумаем, где мы можем взять данные для того, чтобы....
- Обучить нейросеть определять тональность высказывания (проще говоря, является ли текст положительным/хвалебным, негативным или нейтральным)
- Создать программу, которая будет определять, написан ли текст в формальном стиле или нет
- Обучить нейросеть определять хейтспич?
- Создать  функцию, которая убирает из текста стоп-слова?
- Создать спеллчекер?

### 1.2 Парсинг

In [ ]:
import urllib.request
import re
import os

In [ ]:
def parcer(url):
   page = urllib.request.urlopen(url)
   text = page.read().decode('utf-8')
   return text

In [ ]:
url = 'https://ru.wikipedia.org/wiki/%D0%9B%D0%B8%D0%BD%D0%B3%D0%B2%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B0'

In [ ]:
text = parcer(url)

In [ ]:
text[:1000]

'<!DOCTYPE html>\n<html class="client-nojs" lang="ru" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Лингвистика — Википедия</title>\n<script>(function(){var className="client-js";var cookie=document.cookie.match(/(?:^|; )ruwikimwclientpreferences=([^;]+)/);if(cookie){cookie[1].split(\'%2C\').forEach(function(pref){className=className.replace(new RegExp(\'(^| )\'+pref.replace(/-clientpref-\\w+$|[^\\w-]+/g,\'\')+\'-clientpref-\\\\w+( |$)\'),\'$1\'+pref+\'$2\');});}document.documentElement.className=className;}());RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\\t.","\xa0\\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","январь","февраль","март","апрель","май","июнь","июль","август","сентябрь","октябрь","ноябрь","декабрь"],"wgRequestId":"28cec913-e6b5-4924-a9ab-9a7f49f0e1d9","wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Лингвистика","wgTitle":"Лингвистика","wgCurRevisionId":139826574,

#### Тренировка на регулярки!
Как можно быстро убрать теги?
Подсказка: сам текст страницы лежит после закрытия тега table.


In [ ]:
def cleanhtml(raw_html):
    # ВАШ Код
    pass

clean_text = cleanhtml(parcer(url))
clean_text

Важно помнить, что каждый отдельный сайт имеет свою структуру, а следовательно, требует отдельных методов обработки. Поэтому, для сохранения нервов и времени, стоит брать сразу такие сайты, в которых есть достаточно данных для дальнейшей работы.

#### Автоматизация: BeautifulSoup

In [ ]:
from bs4 import BeautifulSoup
text = parcer(url)
soup = BeautifulSoup(text, 'html.parser')
print(soup.text.replace('\n\n\n', ' ')[862:1200])

Лингви́стика (от лат. lingua «язык»), языкозна́ние, языкове́дение — наука, изучающая язык[1][2].
Это наука о естественном человеческом языке вообще и обо всех языках мира как его индивидуализированных представителях.
В широком смысле слова лингвистика подразделяется на научную и практическую. Чаще всего под лингвистикой подразумевается 


#### Автоматизация: Wikipedia
Многие крупные страницы уже имеют решения по автоматической обработке данных

In [ ]:
import wikipedia as wpd

In [ ]:
li = wpd.page("Linguistics")
print(li.title)
print(li.url)

Linguistics
https://en.wikipedia.org/wiki/Linguistics


In [ ]:
wpd.set_lang("ru")
ly = wpd.page("Лингвистика")

ly.links[:10]

['Gemeinsame Normdatei',
 'Акустика',
 'Акустика речи',
 'Анкетирование в лингвистике',
 'Антропологическая лингвистика',
 'Антропонимика',
 'Арго',
 'Ареалогия',
 'Ареальная лингвистика',
 'Артикуляционная фонетика']

In [ ]:
clean_text = ly.content
print(clean_text[:470])

Лингви́стика (от лат. lingua «язык»), языкозна́ние, языкове́дение — наука, изучающая язык.
Это наука о естественном человеческом языке вообще и обо всех языках мира как его индивидуализированных представителях.
В широком смысле слова лингвистика подразделяется на научную и практическую. Чаще всего под лингвистикой подразумевается именно научная лингвистика; профессионально ей занимаются учёные-лингвисты. Лингвистика связана с семиотикой как наукой о знаковых система


__Вопрос на подумать:__

> Как из имеющихся команд можно быстро собрать большое количество, похожих по теме статей?

---



In [ ]:
from tqdm import tqdm

def get_similar_texts(lang, page_name, num=10):
    wiki_content = []
    wpd.set_lang(lang)
    pbar = tqdm(total=num)

    page = wpd.page(page_name)
    wiki_content.append("%s\n%s" % (page.title, page.content.replace('=', '')))
    pbar.update(1)

    i = 0
    while (len(wiki_content) != num) and (i < len(wiki_content)):
        page_name = page.links[i]
        try:
            page = wpd.page(page_name)
            wiki_content.append("%s\n%s" % (page.title, page.content.replace('=', '')))
            pbar.update(1)
        except wpd.exceptions.WikipediaException:
            print("Skip %s" % page_name)
        i += 1

    pbar.close()
    return wiki_content

In [ ]:
similar_texts = get_similar_texts('ru', 'Лингвистика')

100%|██████████| 10/10 [00:24<00:00,  2.45s/it]


In [ ]:
# similar_texts[:2]

На семинарах мы, как правило, будем работать на относительно небольшых объёмах данных -- так что сейчас мы не будем скачивать большой текст для нашего неумеющего в морфологию спеллчекера, а поработаем на относительно небольшом объёме данных -- на нашем тексте про лингвистику.

## 2. Препроцессинг

Существуют различные виды препроцессинга, подходящие для разных видов задач:

1. Токенизация
2. Нормализация/лемматизация
3. POS-tagging
4. Чанкинг
5. Словари и множества для хранения данных

А ещё синтаксические парсеры, деревья, поиск именованных сущностей и тд.

### 2.1 Токенизация

Какой может быть токенизация? Предложите свой вариант токенизации в ячейке ниже.

In [ ]:
from string import punctuation

In [ ]:
sent = 'Из-под стола выглянул рыжий кот, я удивился'

Обычно используется NLTK для токенизации

In [ ]:
tokens = nltk.word_tokenize(sent)
tokens[:10]

In [ ]:
tokens = nltk.wordpunct_tokenize(sent)
tokens[:10]

### 2.2 Лемматизация

Разным языкам - разные лемматизаторы. Да и сами лемматизаторы разные:
- pymorphy: большой словарь, которые подбирает форму слова на основе высчитанной статистики.
    
    \+ работает быстро

    \- не всегда верно, так как не опирается на контекст
- mystem: тоже основано на словарях, но при этом умеет в контекст и ещё может анализировать неизвестные слова на основе известной ей морфологии

    \+ умеет снимать однозначность

    \- работает медленнеее (хотя на больших текстах может и быстрее)

И ещё много-много-много, но эти __база__.

In [ ]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer(lang='ru')

In [ ]:
sent = 'Для безе надо взбить белки'
tokens_pymorphy = nltk.word_tokenize(sent)
tokens_pymorphy[:10]

['Для', 'безе', 'надо', 'взбить', 'белки']

In [ ]:
for token in tokens_pymorphy[:10]:
    print(token, morph.parse(token)[0].normal_form, sep='\t')

Для	для
безе	безе
надо	надо
взбить	взбить
белки	белка


In [ ]:
from pymystem3 import Mystem
mystem = Mystem()

In [ ]:
lemmas = mystem.lemmatize(sent)
print(sent)
print(''.join(lemmas))

Для безе надо взбить белки
для безе надо взбивать белок



### 2.3 POS-tagging

Например, NLTK

In [ ]:
sentence = "At eight o'clock on Thursday morning Arthur didn't feel very good."
tokens = nltk.word_tokenize(sentence)
tagged = nltk.pos_tag(tokens)
tagged

[('At', 'IN'),
 ('eight', 'CD'),
 ("o'clock", 'NN'),
 ('on', 'IN'),
 ('Thursday', 'NNP'),
 ('morning', 'NN'),
 ('Arthur', 'NNP'),
 ('did', 'VBD'),
 ("n't", 'RB'),
 ('feel', 'VB'),
 ('very', 'RB'),
 ('good', 'JJ'),
 ('.', '.')]

Но только на английском... Какие еще постеггеры вы знаете?

In [ ]:
!python3 -m spacy download ru_core_news_sm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 48.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
nlp = spacy.load('ru_core_news_sm')

doc = nlp(sent)
for token in doc:
    print(token.text, token.pos_)

Для ADP
безе NOUN
надо ADV
взбить VERB
белки NOUN


### 2.4 Chunking и более сложная синтаксическая обработка

![picture](https://drive.google.com/uc?export=view&id=1bgUdg7vqbU4PgqiNoHbX81cz1fmIva5e)


https://drive.google.com/file/d/1bgUdg7vqbU4PgqiNoHbX81cz1fmIva5e/view?usp=sharing

Давайте при думаем правила простого чанкинга линейных именных групп в английском посредством nltk

In [ ]:
sentence = [
    ("the", "DT"), ("little", "JJ"), ("yellow", "JJ"), ("dog", "NN"),
    ("barked", "VBD"), ("at", "IN"),  ("the", "DT"), ("cat", "NN")
]
grammar = "ВАШЕ ПРАВИЛО"

cp = nltk.RegexpParser(grammar)
result = cp.parse(sentence)
print(result)

(S
  (NP the/DT little/JJ yellow/JJ dog/NN)
  barked/VBD
  at/IN
  (NP the/DT cat/NN))


Можно ли для русского синтаксиса придумать подобные правила на основе частей речи? Возьмем предложение:
"Богатый мужчина дал золотых монет бедному"

1. Давайте попробуем разделить его на чанки.
2. В каком формате это можно выводить?
3. Напишите на основе наших размышлений код.

In [ ]:
sent = 'Богатый мужчина дал золотых монет бедному'

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
p = morph.parse('стали') [0]
p.tag
#p.tag.case
#p.tag.gender
#p.tag.number
#p.tag.POS

OpencorporaTag('VERB,perf,intr plur,past,indc')

In [ ]:
ent = ('NOUN', 'ADJF', 'PRTF')

def chunking(sent):
    chunked_sent = []
    chunk = []

    # ВАШ КОД

    return chunked_sent


In [ ]:
chunked_sent = chunking(sent.split())
chunked_sent

[['Богатый', 'мужчина'], 'дал', ['золотых', 'монет'], ['бедному']]

Однако, что делать с различного роди причастными оборотами или предложениями с "который"? Тут уже не обойтись без деревьев. В python есть библиотеки типа UD_Pipe, которые рисуют готовое дерево - мы рассмотрим их подробнее на следующих занятиях по теме.

> Под "чанкингом" также часто имеют в виду разделение текста (или аудио, например) на куски, чтобы было проще обучать нейронные сети. Например, у BERT'а ограничен размер входного сообщения до 512 токенов, поэтому для обработки очень больших текстов требуется разделить его на куски, т.е. чанки



### 2.4 Словари и сеты

1. В чём принципиальная разница между множеством (сетом) и массивом из уникальных элементов?
2. Существуют ли сценарии при которых нужно использовать массив из уникальных элементов?

In [ ]:
clean_text = clean_text.lower()

no_accent = ['а', 'я', 'у', 'ю', 'о', 'е', 'э', 'и', 'ы', 'А', 'Я', 'У', 'Ю', 'О', 'Е', 'Э', 'И']
accented = ['а́', 'я́', 'у́', 'ю́', 'о́', 'е́', 'э́', 'и́', 'ы́', 'А́', 'Я́', 'У́', 'Ю́', 'О́', 'Е́', 'Э́', 'И́']

for acc, no_acc in zip(accented, no_accent):
    clean_text = clean_text.replace(acc, no_acc)

['Лингви́стика',
 'от',
 'лат',
 'lingua',
 'язык',
 'языкозна́ние',
 'языкове́дение',
 'наука',
 'изучающая',
 'язык']

In [ ]:
tokens = nltk.word_tokenize(clean_text)
tokens = [token.strip(punctuation) for token in tokens if token.strip(punctuation)!='']
tokens[:10]

2027

In [ ]:
from collections import Counter

WORDS = Counter()
WORDS.update(tokens)

In [ ]:
# WORDS

## 3. Анализ и работа с данными

Для работы с данными нет единого механизма - тут могут пригодиться и нейросети, и правила, стороннние библиотеки, и много чего еще - в рамках данного занятия посмотрим на вектора и расстояние Левенштейна.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from collections import Counter
import textdistance

### Векторные методы

1. Какие методы преобразования тексты в вектора (векторайзеры) вы знаете?
2. Можете описать как работает векторайзер?
3. Как сравнить насколько вектора похожи?

In [ ]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

In [ ]:
vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

def get_closest_match_vec(text, X, vec, TOPN=5):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]

    return [id2word[top] for top in topn]

get_closest_match_vec('асновного', X, vec)

['основного', 'основана', 'иностранного', 'оно', 'но']

### Расстояние Левенштейна

1. Что такое расстояние Левенштейна?
2. Какие у него плюсы и минусы?

![Изображение с ресурса "Системный Блокъ"](https://sysblok.ru/wp-content/uploads/2020/12/image1-2.png)


In [ ]:
def get_closest_match_with_metric(text, X, vec, metric=textdistance.levenshtein):
    similarities = Counter()
    lookup = get_closest_match_vec(text, X, vec, TOPN=3)
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word)

    return similarities.most_common(1)[0]

In [ ]:
get_closest_match_with_metric('ленгвистика', X, vec, textdistance.hamming)

('лингвистика', 0.9090909090909091)

In [ ]:
get_closest_match_with_metric('импирическая', X, vec, textdistance.hamming)

('Эмпирическая', 0.9166666666666666)

In [ ]:
get_closest_match_with_metric('мадэлей', X, vec, textdistance.hamming)

('моделей', 0.7142857142857143)

При выборе модели решения задачи мы всегда осуществляем некий трейд-офф между эффективность и время- энерго- затратностью. Алгоритмы, которые работают "качественнее"/ "эффективнее" часто требуют больших мощностей или времени.

А как кстати определить качество? И как его улучшить?

### 4. Качество

Для оценки качества подхода требуется большое количество данных и правильно подобранные метрики. Базовые метрики: __accuracy, precision, recall, f1__.

![picture](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTZnX2JPyF9icH7XMlM4BIlgw74TH4yWNG3kA&s)

Если картинка не открывается, она тут: https://drive.google.com/uc?export=view&id=1ibvBKRAy_mn-ln4iZfzJNFTm349HCTmT

## $F_{1}=\frac{2\times\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}$

## $Accuracy=\frac{\text{TP} + \text{TN}}{\text{TP} + \text{TN} + \text{FP} + \text{FN}}$

Можно ли найти уже готовый текст, на котором мы можем проверить наш спеллчекер?

На большой коллекции текстов с сайтов интернет-газет и размеченных тектов диалога получились следующие метрики:
- precision ~ 40%,
- recall ~ 60%

Что значат эти показатели с точки зрения работы модели и имеющихся данных? Как улучшить эти показатели?

Давайте посмотрим на ошибки.

Так, среди ошибок

['симпатичнейшое', 'пояним', 'полчатся', 'нащщот', 'основая', 'вобщем', 'как', 'вы', 'знаете', 'из', 'моего', 'не', 'давнего', 'ящека', 'хороше', 'патаму', 'шта', 'поффтыкав', 'соре', 'чтото', 'мошный', 'хороше', 'седня', 'вешать', 'эт', 'канешна', 'начальнег', 'павзрослому', 'подсаживаеться', 'какието', 'малолетки', 'монголоидом', 'баръер', 'коментариев', 'отвественный', 'расчитаны', 'обълись', 'распрашивая', 'зубодробительня', 'каааак', 'расщифровать', 'самойто', 'тул', 'ваще', 'тока', 'навернео', 'ооочень', 'както', 'мущщину', 'никада']

In [ ]:
# get_closest_match_with_metric('выигрышь', X, vec, textdistance.hamming)